In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('/content/urldatapre.csv')

In [3]:
df.head()

,Unnamed: 0,url,label,result
0,0,https www google com,benign,0.0
1,1,https www youtube com,benign,0.0
2,2,https www facebook com,benign,0.0
3,3,https www baidu com,benign,0.0
4,4,https www wikipedia org,benign,0.0


In [4]:
###Drop Nan Values
df=df.dropna()

In [5]:
X=df.drop('label',axis=1)

In [6]:
y=df['result']

In [7]:
X.shape

(386048, 3)

In [8]:
y.shape

(386048,)

In [9]:
import tensorflow as tf

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
### Vocabulary size
voc_size=5000

In [12]:
messages=X.copy()

In [13]:
messages['url'][1]

'https www youtube com'

In [14]:
messages.reset_index(inplace=True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['url'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
381048
381049
381050
381051
381052
381053
381054
381055
381056
381057
381058
381059
381060
381061
381062
381063
381064
381065
381066
381067
381068
381069
381070
381071
381072
381073
381074
381075
381076
381077
381078
381079
381080
381081
381082
381083
381084
381085
381086
381087
381088
381089
381090
381091
381092
381093
381094
381095
381096
381097
381098
381099
381100
381101
381102
381103
381104
381105
381106
381107
381108
381109
381110
381111
381112
381113
381114
381115
381116
381117
381118
381119
381120
381121
381122
381123
381124
381125
381126
381127
381128
381129
381130
381131
381132
381133
381134
381135
381136
381137
381138
381139
381140
381141
381142
381143
381144
381145
381146
381147
381148
381149
381150
381151
381152
381153
381154
381155
381156
381157
381158
381159
381160
381161
381162
381163
381164
381165
381166
381167
381168
381169
381170
381171
381172
381173
381174
381175
381176
381177
381178
381179
381180
381181
381182
3811

In [18]:
corpus[5656]

'http www avalanch nhl com index html'

In [19]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[2644, 3809, 4748, 147],
 [2644, 3809, 3920, 147],
 [2644, 3809, 2325, 147],
 [2644, 3809, 3578, 147],
 [2644, 3809, 4602, 2711],
 [2644, 3809, 4546, 147],
 [2644, 3809, 1011, 147],
 [2644, 3809, 4748, 2770],
 [2644, 3809, 2047, 147],
 [2644, 3809, 881, 147],
 [2644, 3809, 1476, 147],
 [2644, 3809, 732, 147],
 [2644, 3809, 458, 147],
 [2644, 3809, 4748, 2770, 2130],
 [2644, 3809, 2206, 147],
 [2644, 3809, 1027, 147],
 [2644, 3809, 4947, 147],
 [2644, 3809, 231, 147],
 [2644, 3809, 2265, 147, 2862],
 [2644, 3809, 1074, 147],
 [2644, 3809, 799, 147],
 [2644, 3809, 2862],
 [2644, 3809, 4748, 2537],
 [2644, 3809, 4748, 2770, 2545],
 [2644, 3809, 4748, 147, 3573],
 [2644, 3809, 4748, 4283],
 [2644, 3809, 4748, 235],
 [2644, 3809, 1614, 147],
 [2644, 3809, 3231, 235],
 [2644, 3809, 1319, 458, 147],
 [2644, 3809, 916, 147],
 [2644, 3809, 3749, 3014],
 [2644, 3809, 4748],
 [2644, 3809, 2770],
 [2644, 3809, 4748, 147, 563],
 [2644, 3809, 4562, 147],
 [2644, 3809, 4748, 4159],
 [2644, 3809, 275

In [20]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3809 4748  147]
 [   0    0    0 ... 3809 3920  147]
 [   0    0    0 ... 3809 2325  147]
 ...
 [   0    0    0 ...  993 3014  111]
 [   0    0    0 ... 1532 1164 3682]
 [   0    0    0 ... 3014  111  993]]


In [21]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0, 2644, 3809, 4748,  147], dtype=int32)

In [22]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [23]:
len(embedded_docs),y.shape

(386048, (386048,))

In [24]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:
X_final.shape,y_final.shape

((386048, 20), (386048,))

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [28]:

model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=1,batch_size=64)

4042/4042 [==============================] - 149s 37ms/step - loss: 0.0180 - accuracy: 0.9950 - val_loss: 0.0174 - val_accuracy: 0.9950
